In [1]:
# if the files are on Anvil / Terra workspace 
# please run it from the workspace analysis to avoid bucket access permission issues
# or use the gcloud login auth interface to authenticate yourself

In [2]:
# Import the methods from the notebook
%run synapse_utils.ipynb

Welcome, eila!



In [3]:
%run tsv_files_utils.ipynb

In [4]:
# TODO: check that you have a list of all your execution ids
execution_id = "0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab"

In [5]:
# NO NEED TO CHANGE!
bucket_id = "fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901"

In [6]:
# read the proc.tsv object
# Create a subset file of each rna_dataset and save it under the rna_dataset and call it {rna_dataset}_metadata.tsv

# TODO: copy the metadata file for ENCODE datasets
if os.path.exists("proc.tsv") == False:
    !gsutil cp 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/ENCODE_synpase_annotation_mapping/proc.tsv' .
else:
    print("file is available, nothing to copy")

file is available, nothing to copy


In [7]:
import csv
from itertools import islice
import pandas as pd
metadata_col_names = None
df_relevant_rows = None

filename = "proc.tsv"  # Replace with the path to your TSV file

with open(filename, "r", newline="") as tsvfile:
    reader = csv.reader(tsvfile, delimiter="\t")  # Set delimiter as tab

# Column names: ['cell_type_name,"rna_dataset","rna_library","rna_barcode","atac_dataset","atac_library","atac_barcode","file"']    # Read the header row
    metadata_col_names = next(reader)
    print("Column names:", metadata_col_names," type is {}".format(type(metadata_col_names)))
    metadata_col_names = metadata_col_names[0].split(",")
    print("metadata_col_names after split is {}".format(metadata_col_names))
    metadata_col_names = [x.strip('"') for x in metadata_col_names]
    print("metadata_col_names after strip is {}".format(metadata_col_names))
    print("Column names after processing is:", metadata_col_names," type is {}".format(type(metadata_col_names)))
    column_atac_dataset_index = metadata_col_names.index("atac_dataset")
    print("column_atac_dataset_index is {}".format(column_atac_dataset_index))

    rows = []  # To store the extracted rows

    # Read only the first 10 lines
    # subset_rows = list(islice(reader, 100000)) 
    # Process the data rows
    # for row in subset_rows: # to do: comment
    for row in reader: # to do: un-comment
        # Process each row
        # Get the values of the desired columns for each row
        # print("row is {}. type is {}. len is {}".format(row,type(row),len(row)))
        row = row[0].split(",")
        row = [x.strip('"') for x in row]
        # print("row after processing is {}. type is {}. len is {}".format(row,type(row),len(row)))
        # print("column_rna_dataset_value is {}".format(column_rna_dataset_value))
        column_atac_dataset_value = row[column_atac_dataset_index]
        # print("column_atac_dataset_value is {}".format(column_atac_dataset_value))
        

        if column_atac_dataset_value != 'NA':
            rows.append(row)
            # print("adding row {}".format(row))
        else:
            # print("Nothing to add: column_atac_dataset_value is {}".format(column_atac_dataset_value))
            continue
    # Create a dataframe from the extracted rows
    df_encode_rows = pd.DataFrame(rows, columns=metadata_col_names)

    # Print the subset dataframe
#     print(df_relevant_rows)


Column names: ['cell_type_name,"rna_dataset","rna_library","rna_barcode","atac_dataset","atac_library","atac_barcode","file"']  type is <class 'list'>
metadata_col_names after split is ['cell_type_name', '"rna_dataset"', '"rna_library"', '"rna_barcode"', '"atac_dataset"', '"atac_library"', '"atac_barcode"', '"file"']
metadata_col_names after strip is ['cell_type_name', 'rna_dataset', 'rna_library', 'rna_barcode', 'atac_dataset', 'atac_library', 'atac_barcode', 'file']
Column names after processing is: ['cell_type_name', 'rna_dataset', 'rna_library', 'rna_barcode', 'atac_dataset', 'atac_library', 'atac_barcode', 'file']  type is <class 'list'>
column_atac_dataset_index is 4


In [8]:
df_encode_rows.head(2)
# ['zR 2', 'ENCSR362YDM', 'ENCLB943KZN', 'GCTTGTTGTTTAACCC', 'ENCSR420EWQ', 'ENCLB006QEF', 'TCGCTAATGAAGCCAA', 'EAID_000003']

,cell_type_name,rna_dataset,rna_library,rna_barcode,atac_dataset,atac_library,atac_barcode,file
0,Hepatocytes,NA,NA,NA,ENCSR114OKU,ENCLB431YMH,GGAGGATCAAGCGGTA,EAID_000002
1,Hepatocytes,NA,NA,NA,ENCSR114OKU,ENCLB431YMH,GTTACGATCGTGGTAT,EAID_000002


In [9]:
len(df_encode_rows['atac_dataset'].to_list())

1236715

In [10]:
!ls gsutil_ls_txt*

gsutil_ls_txt.txt


In [11]:
# LOAD the ls file from local - does not work on this machine
def load_gs_ls_text_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        lines = [line.strip() for line in lines]
    return lines

# Example usage
file_path = 'gsutil_ls_txt.txt'
gs_path_to_search_sample_id = load_gs_ls_text_file(file_path)
print(gs_path_to_search_sample_id[0:5])


['gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/8ef75ac6-1a49-4cbe-b61d-d62af4c904d7/share/24e3d191-38aa-4ed1-9255-dbfb14161097/call-atac/wf_atac/3d2abad2-1ff0-4477-a7a8-35aef31e2a3e/call-align/ENCSR618WVK.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/c467e890-b9f7-4458-a5b7-5581eb958cba/share/293acd9b-1e89-41d4-af76-247ca41af146/call-atac/wf_atac/2c759a0e-018b-4d6e-bbfc-257cbab95224/call-align/ENCSR868ZEI

In [16]:
gs_atac_dataset = [x.split("/")[-1].split(".")[0] for x in gs_path_to_search_sample_id]

In [17]:
atac_dataset_list = df_encode_rows["atac_dataset"].tolist()
atac_dataset_list[0:3]

['ENCSR114OKU', 'ENCSR114OKU', 'ENCSR114OKU']

In [18]:
# atac dataset in both
atac_dataset_to_fragment = list(set(gs_atac_dataset) & set(atac_dataset_list))
atac_dataset_to_fragment[0:6]

['ENCSR776EOI',
 'ENCSR455CVZ',
 'ENCSR660NEE',
 'ENCSR453TVZ',
 'ENCSR302CMX',
 'ENCSR411MLR']

In [19]:
len(atac_dataset_to_fragment)

23

In [20]:
df_relevant_rows = df_encode_rows[df_encode_rows['atac_dataset'].isin(atac_dataset_to_fragment)]
df_relevant_rows

,cell_type_name,rna_dataset,rna_library,rna_barcode,atac_dataset,atac_library,atac_barcode,file
510368,macrophage,ENCSR231FNL,ENCLB398IAZ,AAACCGCGTGACATAT,ENCSR987PQH,ENCLB714RNW,GTCACCATGAGCTATT,EAID_000093
510369,cardiomyocyte,ENCSR231FNL,ENCLB398IAZ,AAACCGCGTTACATCC,ENCSR987PQH,ENCLB714RNW,ATGTTGTTGAGCTATT,EAID_000093
510370,pericyte,ENCSR231FNL,ENCLB398IAZ,AAACCGCGTTTATTCG,ENCSR987PQH,ENCLB714RNW,CACCATATGAGCTATT,EAID_000093
510371,cardiomyocyte,ENCSR231FNL,ENCLB398IAZ,AAACGCGCAGCATGTC,ENCSR987PQH,ENCLB714RNW,CTAAGTAACGGGTTAT,EAID_000093
510372,pericyte,ENCSR231FNL,ENCLB398IAZ,AAACGGATCCATAAGC,ENCSR987PQH,ENCLB714RNW,CAAGCTCCTAGGCCTC,EAID_000093
...,...,...,...,...,...,...,...,...
1225635,Vascular_smooth_muscle_cell,NA,NA,NA,ENCSR455CVZ,ENCLB859JTQ,CATGAACGTGGAAGTCGTAGTA,EAID_000074
1225636,Vascular_smooth_muscle_cell,NA,NA,NA,ENCSR455CVZ,ENCLB859JTQ,CGCATCTCTGCTGGTAGCGTAT,EAID_000074
1225637,Vascular_smooth_muscle_cell,NA,NA,NA,ENCSR455CVZ,ENCLB859JTQ,GCTGATCTTCTCGCTTAGCCTT,EAID_000074
1225638,Vascular_smooth_muscle_cell,NA,NA,NA,ENCSR455CVZ,ENCLB859JTQ,CCTCGATATGCGCGATACAAAT,EAID_000074


In [21]:
# strings = gs_path_to_search_sample_id
# substrings = atac_dataset_to_fragment

gs_path_to_fragment = list(filter(lambda x: any(substring in x for substring in atac_dataset_to_fragment), gs_path_to_search_sample_id))

print(gs_path_to_fragment[0:4])


['gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/97de835d-0f4e-474e-a583-987d6f6a4f2c/share/d8870ad0-1b10-454a-8c36-75350ea50cbb/call-atac/wf_atac/c53f4afa-8fd4-4995-93e1-7f4dee359afc/call-align/ENCSR023FME.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/c467e890-b9f7-4458-a5b7-5581eb958cba/share/293acd9b-1e89-41d4-af76-247ca41af146/call-atac/wf_atac/2c759a0e-018b-4d6e-bbfc-257cbab95224/call-align/ENCSR868ZEI.atac.filter.fragments.hg38.tsv.gz', 'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/ff3a5809-0cd2-464c-8a29-9bf6311111ae/share/02c2df11-8d6c-4786-868a-94ecf3858bc5/call-atac/wf_atac/26b48087-2f17-469d-b3ff-b64720f9ddb4/call-align/ENCSR367GKP

In [22]:
len(gs_path_to_fragment)

23

In [23]:
gs_path_to_fragment[0]

'gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz'

In [24]:
# os.path.basename("gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz")

df_relevant_rows[df_relevant_rows['atac_dataset']=='ENCSR987PQH']['file'].to_list()[0]
                 

'EAID_000093'

In [ ]:
%run tsv_files_utils.ipynb
for fragment_file in gs_path_to_fragment[0:1]:
    print("fragment_file is {}".format(fragment_file))

    # TODO: uncomment on the cloud
    # !gsutil cp $fragment_file - | gzip -dc | head -5
    # rename the file to: ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz
    local_file_name = os.path.basename(fragment_file)
    print("local_file_name is {}".format(local_file_name))
    print(os.path.exists(local_file_name))
    # copy the file locally. if not already exists
    if (os.path.exists(local_file_name) == False):  
      !gsutil cp $fragment_file .
    else:
      print("Nothing to copy. all files available")
    # Define column names
    column_names = ['chr', 'start_pos', 'end_pos', 'cell_id_atac_dataset' ,'read_count']
    # file_path,column_names
    full_local_file_name = os.path.join(os.getcwd(),local_file_name)
    print("full_local_file_name is {}".format(full_local_file_name))
    print("read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines_with_col_names for file is {} and column_names {}".format(full_local_file_name,column_names))
    df = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines_with_col_names(full_local_file_name,column_names)
    
    # print("calling split_column_to_cell_id_and_atac_dataset for file {}. df is {}".format(full_local_file_name,df))
    df_split = split_column_to_cell_id_and_atac_dataset(df)
    # Create an instance of ColumnSplitter
    # splitter = ColumnSplitter(column_name='cell_id_atac_dataset')

    # Split the column and create new columns
    # df_splitter = splitter.split_column(df)
    
    # Identify the unique values in the specific column
    unique_barcode_values_on_fragments = df['atac_dataset'].unique()
    print("number of files to create is {}".format(unique_barcode_values_on_fragments))

    cur_atac_dataset = fragment_file.split("/")[-1].split(".")[0]
    print("cur_atac_dataset is {}".format(cur_atac_dataset))
    # get the folder name in Synpase
    folder_name = df_relevant_rows[df_relevant_rows['atac_dataset']==cur_atac_dataset]['file'].to_list()[0]
    print("folder_name in synpase is {}".format(folder_name))
    dataset_id = 'syn34271785'
    # (starting_folder_id, first_subfolder_name, second_subfolder_name):
    file_id_list = get_file_names_in_second_subfolder(dataset_id,folder_name,"labels")
    # get_label_file_by_folder_name(dataset_ids,folder_name)
    print("file_id_list is {}".format(file_id_list))

    #file_handle_id, destination_path
    file_id = file_id_list[0] # the label file synID
    print('file_id is {}'.format(file_id))
    
    # Obtain a pointer and download the data 
    label_file_synHandle = syn.get(entity=file_id) 
    print('label_file_synHandle is {}'.format(label_file_synHandle))
     # Get the path to the local copy of the data file 
    full_labels_file_path = label_file_synHandle.path
    print('full_labels_file_path is {}'.format(full_labels_file_path))

    df_labels = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_labels_file_path)
    print("df_labels is {}".format(df_labels))
    # df,fld_name,col_name,prefix_file_name)
    split_files_dir = os.path.join(os.getcwd(),cur_atac_dataset)
    label_file_to_cp = split_df_based_on_col_name(df_labels,split_files_dir,"cell_type_id",cur_atac_dataset)
    print("finished split_df_based_on_col_name. label_file_to_cp are {}".format(label_file_to_cp))
    
    # TODO: check & copy the local folder to the bucket
    gs_dest = os.path.dirname(fragment_file)
    print("copy the local folder {} to gcp  {}".format(split_files_dir,gs_dest))
    # !gsutil -m cp split_files_dir $gs_dest

fragment_file is gs://fc-secure-0a879173-62d3-4c3a-8fc3-e35ee4248901/submissions/0e8c4f76-5f9f-4dc8-8ef3-608afb9395ab/share/38144a43-b98a-4c90-93ff-a7fbb88efd2f/call-atac/wf_atac/e7b3e84a-fea9-4fe6-ad7a-40af393fdb36/call-align/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz
local_file_name is ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz
True
Nothing to copy. all files available
full_local_file_name is /users/eila/git_kundaje/DNA/ATAC/single-cell/fragments-file-manipulation/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz
read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines_with_col_names for file is /users/eila/git_kundaje/DNA/ATAC/single-cell/fragments-file-manipulation/ENCSR987PQH.atac.filter.fragments.hg38.tsv.gz and column_names ['chr', 'start_pos', 'end_pos', 'cell_id_atac_dataset', 'read_count']


In [ ]:
# !rm -r ENCSR987PQH

In [ ]:
# !ls ENCSR987PQH/cardiomyocyte

In [ ]:
# !cat ENCSR987PQH/cardiomyocyte/split_fragment_file_cardiomyocyte.tsv | more

In [ ]:
# df = read_tsv_gz_to_dataframe_skipping_comments_and_empty_lines(full_local_file_name)